### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
player_count=pd.DataFrame({"Total Player Count":[purchase_data['SN'].nunique()]})
player_count

,Total Player Count
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# Calculating number of unique items
nmb_unique_items=purchase_data['Item Name'].nunique()
# nmb_unique_items

In [4]:
# Pull out Average price from .describe() output
avg_price=round(purchase_data.describe()['Price'][1],2)
#  or run .mean() f-n
avg_price=round(purchase_data['Price'].mean(),2)
# avg_price


In [5]:
# Calculating total number of purchases
total_nmb_purchases=purchase_data.shape[0]
#  or
total_nmb_purchases=purchase_data.count()[0]
# total_nmb_purchases

In [6]:
# Calculate total revenue as a sum of Price column
total_revenue=round(purchase_data['Price'].sum(),2)
# total_revenue

In [7]:
# Create header for the summary data frame
purch_smry_cols=['Number of Unique Items', 'Average Purchase Price', 'Total Number of Purchases', 'Total Revenue']
#  Create a two-dimensional list containing the summary data
purch_smry_data=[[nmb_unique_items, avg_price, total_nmb_purchases, total_revenue]]
#  Pass headers and summary data to a pandas dataframe function, assigning single row name to "All data" 
#  by altering default Indexing
purch_smry_df = pd.DataFrame(purch_smry_data, index=[f"All data"], columns=purch_smry_cols)

# or alternatively use Dictionary of Lists
#  Create a dictionary
purch_smry_dic={
            'Number of Unique Items':[nmb_unique_items], 
            'Average Purchase Price':[avg_price], 
            'Total Number of Purchases':[total_nmb_purchases], 
            'Total Revenue':[total_revenue]}
#  Convert dictionary to a Data Frame
purch_smry_df=pd.DataFrame(purch_smry_dic)
#  Add currency formatting
purch_smry_df["Average Purchase Price"]=purch_smry_df["Average Purchase Price"].map("${:.2f}".format)
purch_smry_df["Total Revenue"]=purch_smry_df["Total Revenue"].map("${:,.2f}".format)
purch_smry_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [8]:
# optimized calculation of gender demographics
gender_grp=(purchase_data.groupby(['Gender']))
gender_smry=pd.DataFrame(gender_grp['SN'].nunique())
gender_smry=gender_smry.rename(columns={'SN':'Total Count'})
gender_smry['Percentage of Players']=(gender_smry['Total Count']*100/gender_smry['Total Count'].sum()).map("{:.2f}%".format)
gender_smry=gender_smry.sort_values(['Total Count'], ascending=False)
gender_smry

# (purchase_data.groupby(['Gender']))['SN'].nunique()

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
purch_smry=[]
purch_smry=pd.DataFrame((purchase_data.groupby('Gender').count())['Purchase ID'])
purch_smry=purch_smry.rename(columns={'Purchase ID':'Purchase Count'})
# Calcumating avg. purchase price per person etc. by gender, by first grouping DF by 'Gender' column, calculating MEAN and selecting 'Price' column for output + formatting
purch_smry['Average Purchase Price']=(pd.DataFrame(purchase_data.groupby(['Gender']).mean()))['Price'].map("${:.2f}".format)
# Calcumating total purchase value by gender, by first grouping DF by 'Gender' column, calculating SUM and selecting 'Price' column for output + formatting
purch_smry['Total Purchase Value']=(pd.DataFrame(purchase_data.groupby(['Gender']).sum()))['Price'].map("${:,.2f}".format)
# Calcumating avg. total purchase per person etc. by gender, by first grouping DF by 'Gender' and user 'SN', 
# then salculating a sum per user within a given gender overall category, then grouping the output by 'Gender'
# and calculating MEAN of total expenditure by user for each gender + formatting
purch_smry['Avg Total Purchase per Person']=(pd.DataFrame(purchase_data.groupby(['Gender','SN']).sum().groupby(['Gender']).mean())['Price']).map("${:,.2f}".format)

purch_smry

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [10]:
# First check min and max age values for range reference
print(purchase_data['Age'].min(), purchase_data['Age'].max())

7 45


In [41]:
# Establishing bins
bins=list(range(5,50+1,5))
bins.remove(45)
print(f"A total of {len(bins)} bins were defined: {bins}")

# Set up group of bin names
groups=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
print(f"A total of {len(groups)} groups were defined: {groups}")

# Make a copy of source dataframe
demgr_smry=purchase_data.copy(deep=True)
demgr_smry.head()

# Create a new Age Range column based on designed binning
demgr_smry['Age Range']=pd.cut(purchase_data['Age'],bins, labels=groups, right=False)
# print(demgr_smry)
grp_demgr_smry=demgr_smry.groupby('Age Range')

# out_demgr_smry=[]
out_demgr_smry=pd.DataFrame(grp_demgr_smry['SN'].nunique())
out_demgr_smry=out_demgr_smry.rename(columns={'SN':'Total Count'})
out_demgr_smry['Percentage of Players']=(
    out_demgr_smry['Total Count']*100/out_demgr_smry['Total Count'].sum()
).map("{:.2f}%".format)

out_demgr_smry

A total of 9 bins were defined: [5, 10, 15, 20, 25, 30, 35, 40, 50]
A total of 8 groups were defined: ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']


,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [37]:
out_demgr_smry2=[]
out_demgr_smry2=pd.DataFrame(grp_demgr_smry['Purchase ID'].count())
out_demgr_smry2['Purchase Count']=pd.DataFrame(grp_demgr_smry2['Purchase ID'].count())

NameError: name 'grp_demgr_smry2' is not defined

In [34]:
out_demgr_smry

,Total Count,Percentage of Players,Purchase Count
Age Range,,,
<10,17,2.96%,23
10-14,22,3.83%,28
15-19,107,18.61%,136
20-24,258,44.87%,365
25-29,77,13.39%,101
30-34,52,9.04%,73
35-39,31,5.39%,41
40+,11,1.91%,12


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

